# Visualization

It this notebook, the holoviz ecosystem of visualization tools is explored. All these packages are quite high level and often depend on other packages. Most of the packages in this ecosystem depend on Holoviz, which we will explore first.

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
# Example data
xs = range(-10,11)
ys = [100-x**2 for x in xs]

In [ ]:
wo_unit = hv.Curve((xs, ys), 
                   ('distance','Horizontal distance'), 
                   ('height','Height above sea level'))

distance = hv.Dimension('distance', label='Horizontal distance', unit='m')
height = hv.Dimension(('height','Height above sea level'), unit='m')
with_unit = hv.Curve((xs, ys), distance, height)

# (using + to compose elements is described in the next guide)
wo_unit + with_unit

## Dimensioned Containers

`HoloMap` is excuted eagerly and can be exported to HTML!

In [ ]:
import numpy as np

from holoviews import opts

hv.extension('bokeh')

opts.defaults(opts.Curve(line_width=1))

In [ ]:
def fm_modulation(f_carrier=220, f_mod=220, mod_index=1, length=0.1, sampleRate=2000):
    sampleInc = 1.0/sampleRate
    x = np.arange(0,length, sampleInc)
    y = np.sin(2*np.pi*f_carrier*x + mod_index*np.sin(2*np.pi*f_mod*x))
    return hv.Curve((x, y), 'Time', 'Amplitude')

f_carrier = np.linspace(20, 60, 3)
f_mod = np.linspace(20, 100, 5)

curve_dict = {(fc, fm): fm_modulation(fc, fm) for fc in f_carrier for fm in f_mod}

kdims = [hv.Dimension(('f_carrier', 'Carrier frequency'), default=40),
         hv.Dimension(('f_mod', 'Modulation frequency'), default=60)]
holomap = hv.HoloMap(curve_dict, kdims=kdims)
holomap.opts(opts.Curve(width=600))

For less interactive, more overlay styles, the following can collapse a `HoloMap`

`NdOverlay` - An n-dimensional container which overlays the elements

`NdLayout` - An n-dimensional container which displays the data in separate plot axes and adds titles for each value

`GridSpace` - A 1D or 2D container which lays out up to two dimensions on a grid.

In [ ]:
grid = hv.GridSpace(holomap)
grid.opts(
    opts.GridSpace(plot_size=75),
    opts.Curve(width=100))

## Dynamic Data
For lazy execution, we can use `DynamicMap`

In [ ]:
hv.extension('matplotlib')

In [ ]:
xvals  = np.linspace(-4, 0, 202)
yvals  = np.linspace(4, 0, 202)
xs, ys = np.meshgrid(xvals, yvals)

def waves_image(alpha, beta):
    return hv.Image(np.sin(((ys/alpha)**alpha+beta)*xs))

waves_image(1,0) + waves_image(1,4)

In [ ]:
dmap = hv.DynamicMap(waves_image, kdims=['alpha', 'beta'])
dmap

range is a built in method, you can also use dymension objects with range

In [ ]:
dmap.redim.range(alpha=(1, 5.0), beta=(1, 6.0))

## Geometry data

The holoviz ecosystem contains multiple software packages. To do projections, holoviews is extended using `geoviews`.

In [ ]:
import xarray as xr
import geoviews as gv
import geoviews.feature as gf

from cartopy import crs
from geoviews import opts

gv.extension('matplotlib')

gv.output(size=150)

In [ ]:
!curl -L http://assets.holoviews.org/geoviews-sample-data.zip -o ./data.zip

In [ ]:
!unzip -o data.zip -d data

In [ ]:
xr_ensemble = xr.open_dataset('data/ensemble.nc').load()
xr_ensemble

create geoviews dataset

In [ ]:
kdims = ['time', 'longitude', 'latitude']
vdims = ['surface_temperature']

xr_dataset = gv.Dataset(xr_ensemble, kdims=kdims, vdims=vdims)

In [ ]:
xr_dataset.to(gv.Image, ['longitude', 'latitude'])

We can also change projection using cartopy

In [ ]:
(xr_dataset.to.image(['longitude', 'latitude']) * gf.coastline).opts(
    opts.Image(projection=crs.Geostationary(), cmap='Greens', xaxis=None, yaxis=None))

## Panel
Panel uses these and other components together with a library called [param](https://param.holoviz.org/) to create dashboard and interactive components.

In [ ]:
import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np

hv.extension('bokeh')

In [ ]:
pn.extension(design='material')

csv_file = ("https://raw.githubusercontent.com/holoviz/panel/main/examples/assets/occupancy.csv")
data = pd.read_csv(csv_file, parse_dates=["date"], index_col="date")

data.tail()

In [ ]:
def view_hvplot(avg, highlight):
    return avg.hvplot(height=600, width=800, legend=False) * highlight.hvplot.scatter(
        color="orange", padding=0.1, legend=False
    )

def find_outliers(variable="Temperature", window=30, sigma=10, view_fn=view_hvplot):
    avg = data[variable].rolling(window=window).mean()
    residual = data[variable] - avg
    std = residual.rolling(window=window).std()
    outliers = np.abs(residual) > std * sigma
    return view_fn(avg, avg[outliers])

In [ ]:
find_outliers(variable='Temperature', window=20, sigma=10)

In [ ]:
variable_widget = pn.widgets.Select(name="variable", value="Temperature", options=list(data.columns))
window_widget = pn.widgets.IntSlider(name="window", value=30, start=1, end=60)
sigma_widget = pn.widgets.IntSlider(name="sigma", value=10, start=0, end=20)

In [ ]:
bound_plot = pn.bind(find_outliers, variable=variable_widget, window=window_widget, sigma=sigma_widget)

In [ ]:
first_app = pn.Column(variable_widget, window_widget, sigma_widget, bound_plot)

first_app